# Active Ind Testing

In [1]:
import numpy as np
import gurobipy as gp
import math
import sympy
import contextlib
import time
import os
from utils import result, METHODS, INF, EPS
from mps_reader_preprocessor import read_mps_preprocess
from polyhedral_model import PolyhedralModel
from polyhedron import Polyhedron

## Read in Polyhedron P

In [2]:
problem_dir = 'netlib_lp_subset'
# problem = 'kb2'
problem = 'adlittle'
# problem = 'angela_test_prob'
# problem = 'network_ex'
# problem = 'gfrd-pnc'
# problem = 'modszk1'
# problem = 'cycle'
# problem = 'cre-c'
mps_fn=os.path.join(problem_dir, problem)
results_dir='results'
max_time=300
sd_method='dual_simplex'
reset=False,
partition_polytope=False
n=0
k=0
spindle=False
spindle_dim=0
n_cone_facets=0
n_parallel_facets=0

## Perform SDCA

In [3]:
### Build Initial Polyhedron from file
print(f'Reading {mps_fn}...')
c, B, d, A, b = read_mps_preprocess(mps_fn)
print('Building polyhedron...')
P = Polyhedron(B, d, A, b, c)
rank_A = np.linalg.matrix_rank(A)

print('Finding feasible solution...')
x_feasible, vbasis, cbasis = P.find_feasible_solution(verbose=False)

x = x_feasible
c=None
verbose=False
method='dual_simplex'
reset=False
max_time=300
first_warm_start=None
save_first_steps=0
problem_name=''

if c is not None:
    P.set_objective(c)

t0 = time.time()
x_current = x
if save_first_steps:
    np.save('solutions/{}_0.npy'.format(problem_name), x_current)      
active_inds = P.get_active_constraints(x_current)
    
pm = P.build_polyhedral_model(active_inds=active_inds, method=method)

if first_warm_start is not None:
    print('Using custom warm start')
    pm.set_solution(first_warm_start)
t1 = time.time()
build_time = t1 - t0
print('Polyhedral model build time: {}'.format(build_time))
    
sub_times = {'sd': [], 'step': [], 'solve': [], 'phase_times': []}
active_inds_it = []
descent_circuits = []
obj_values = []
step_sizes = []
iter_times = []
simplex_iters = []
iteration = 0
active_inds_it.append(active_inds)
obj_value = P.c.dot(x_current)
obj_values.append(obj_value)
t2 = time.time()
iter_times.append(t2 - t1)


# compute steepest-descent direction
descent_direction, y_pos, y_neg, steepness, num_steps, solve_time, phase_times = pm.compute_sd_direction(verbose=verbose)
# orig_test_dir = descent_direction
# print(descent_direction)
# print(np.count_nonzero(np.array(descent_direction)))
simplex_iters.append(num_steps)
sub_times['solve'].append(solve_time)
sub_times['phase_times'].append(phase_times)
    
t3 = time.time()
sub_times['sd'].append(t3 - t2)
restrict_switches = []
restrict_switches.append(False)
init_inds = None
while abs(steepness) > EPS:
        
    t3 = time.time()
    if reset:
        pm.reset()
        
    # take maximal step
    x_current, alpha, active_inds = P.take_maximal_step(descent_direction, y_pos, y_neg)

    if len(active_inds) < (P.n - rank_A)-math.floor(P.n/10):
        restrict_switches.append(True)
    else:
        restrict_switches.append(False)
    
    if iteration % 50 == 0 or iteration == 1:
        print('\nIteration {}'.format(iteration))
        print('Objective: {}'.format(obj_value))
        print('Steepness: {}'.format(steepness))
        print('Step length: {}'.format(alpha))
        # print('Descent Direction: {}'.format(descent_direction))
        
    t4 = time.time()
    active_inds_it.append(active_inds)
    obj_value = P.c.dot(x_current)
    obj_values.append(obj_value)
    iter_times.append(t4 - t1)
    sub_times['step'].append(t4 - t3) 
    descent_circuits.append(descent_direction)
    step_sizes.append(alpha)     
                
    if math.isinf(alpha):
        # problem is unbounded
        result(status=1, circuits=descent_circuits, steps=step_sizes)
        
    # compute steepest-descent direction

    print(f'Number of Active Inds for Iteration {iteration}: {len(active_inds)}')
    print(f'Switch to Facet Restriction for Iteration {iteration}: {restrict_switches[-1]}')
    
    if restrict_switches[-1] and not restrict_switches[-2]:
        init_inds = active_inds
        print(f'initial inds for switch: {init_inds}')

    if restrict_switches[-1]:
        print('using init_inds')
        pm.set_active_inds(init_inds, simp_switch = restrict_switches[-1])
    else:
        print('using active_inds')
        pm.set_active_inds(active_inds, simp_switch = restrict_switches[-1])

    print('---------------------------------------------------------------')
    # for ind in active_inds:
    #     if abs(P.B_x_current[ind]-P.d[ind]) > EPS:
    #         print(f'Not truly active inds for inex {ind}: {P.B_x_current[ind]} != {P.d[ind]}')
    #     # print(f'check that (Bx)_i = d_i for index {ind}: {P.B_x_current[ind]} = {P.d[ind]}; {abs(P.B_x_current[ind]-P.d[ind])<= EPS}')
    descent_direction, y_pos, y_neg, steepness, num_steps, solve_time, phase_times = pm.compute_sd_direction(verbose=verbose)
        
    t5 = time.time()
    sub_times['sd'].append(t5 - t4)
    sub_times['solve'].append(solve_time)
    sub_times['phase_times'].append(phase_times)
    simplex_iters.append(num_steps)
        
    iteration += 1
    current_time = t5 - t1
    if current_time > max_time:
        result(status=2)
    if iteration <= save_first_steps:
        np.save('solutions/{}_{}.npy'.format(problem_name, iteration), x_current)

t6 = time.time()
total_time = t6 - t1 
print(f'final iteration steepness: {steepness}')
print('Total time for steepest-descent scheme: {}'.format(total_time))

Reading netlib_lp_subset\adlittle...
Building polyhedron...
Problem size: n = 97,  m_B = 138,  m_A = 15
Finding feasible solution...
Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-01
Building polyhedral model. Solve method: dual_simplex
Set parameter Method to value 1
Polyhedral model built!
Polyhedral model build time: 0.010600566864013672

Iteration 0
Objective: 671734.1100278501
Steepness: -1093.3333333333335
Step length: 1.492387497139124
using active_inds
Number of Active Inds for Iteration 0: 93
Switch to Facet Restriction for Iteration 0: False
---------------------------------------------------------------

Iteration 1
Objective: 670102.4330309781
Steepness: -608.695652173913
Step length: 14.047128014198497
using active_inds
Number of Active Inds for Iteration 1: 90
Switch to Facet Restriction for Iteration 1: False
---------------------------------------------------------------
using active_inds
Number of Active Inds for Iteration 2: 90

In [4]:
print(P.n)
print(rank_A)
print(P.m_B)
print(math.floor(P.n/10))

97
15
138
9


In [5]:
print(obj_values[-1])
# print(x_current)

224902.2744477104


In [6]:
print('\nSolving with simplex method...')
lp_result = P.solve_lp(verbose=False, record_objs=True)
print('\nSolution using simplex method:')
print(lp_result)


Solving with simplex method...

Solution using simplex method:

Optimal objective: 225494.96316238024
Total solve time: 0.009578466415405273
Number of iterations: 100.0


In [7]:
equality = np.all(np.isclose(np.dot(A, np.array(x_feasible)), b))
inequality = np.all(np.dot(B, np.array(x_feasible))<= d)
print(equality)
print(inequality)

True
True
